Este notebook tem o primeiro contatocom os dados, e nele iremos fazer uma EDA e explorar os dados

# Setting up

## imports

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from ydata_profiling import ProfileReport
from pathlib import Path

# current repo path 
repo_path = Path().resolve().parent

spark = SparkSession.builder.appName('Spark Demo').master('local[*]').getOrCreate()


## carregando dados

In [19]:
offers = spark.read.json((repo_path / 'data' / 'raw' / 'offers.json').as_posix())
transactions = spark.read.json((repo_path / 'data' / 'raw' / 'transactions.json').as_posix())
profiles = spark.read.json((repo_path / 'data' / 'raw' / 'profile.json').as_posix())

In [20]:
offers.show(10,False)
offers.printSchema()

+----------------------------+--------------+--------+--------------------------------+---------+-------------+
|channels                    |discount_value|duration|id                              |min_value|offer_type   |
+----------------------------+--------------+--------+--------------------------------+---------+-------------+
|[email, mobile, social]     |10            |7.0     |ae264e3637204a6fb9bb56bc8210ddfd|10       |bogo         |
|[web, email, mobile, social]|10            |5.0     |4d5c57ea9a6940dd891ad53e9dbe8da0|10       |bogo         |
|[web, email, mobile]        |0             |4.0     |3f207df678b143eea3cee63160fa8bed|0        |informational|
|[web, email, mobile]        |5             |7.0     |9b98b8c7a33c4b65b9aebfe6a799e6d9|5        |bogo         |
|[web, email]                |5             |10.0    |0b1e1539f2cc45b7b9fa7c272da2e1d7|20       |discount     |
|[web, email, mobile, social]|3             |7.0     |2298d6c36e964ae4a3e7e9706d1fb8c2|7        |discoun

In [21]:
profiles.show(10,False)
profiles.printSchema()

+---+-----------------+------+--------------------------------+-------------+
|age|credit_card_limit|gender|id                              |registered_on|
+---+-----------------+------+--------------------------------+-------------+
|118|NULL             |NULL  |68be06ca386d4c31939f3a4f0e3dd783|20170212     |
|55 |112000.0         |F     |0610b486422d4921ae7d2bf64640c50b|20170715     |
|118|NULL             |NULL  |38fe809add3b4fcf9315a9694bb96ff5|20180712     |
|75 |100000.0         |F     |78afa995795e4d85b5d9ceeca43f5fef|20170509     |
|118|NULL             |NULL  |a03223e636434f42ac4c3df47e8bac43|20170804     |
|68 |70000.0          |M     |e2127556f4f64592b11af22de27a7932|20180426     |
|118|NULL             |NULL  |8ec6ce2a7e7949b1bf142def7d0e0586|20170925     |
|118|NULL             |NULL  |68617ca6246f4fbc85e91a2a49552598|20171002     |
|65 |53000.0          |M     |389bc3fa690240e798340f5a15918d5c|20180209     |
|118|NULL             |NULL  |8974fc5686fe429db53ddde067b88302|2

Vemos que as datas estão desformatadas, irei transformar a coluna no tipo certo

In [22]:
profiles = profiles.withColumn('registered_on', 
                               F.to_date(F.col('registered_on').cast('string'),'yyyyMMdd')
                               )


In [27]:
transactions.show(10,False)
transactions.printSchema()

+--------------------------------+--------------+---------------------+----------------------------------------------------+
|account_id                      |event         |time_since_test_start|value                                               |
+--------------------------------+--------------+---------------------+----------------------------------------------------+
|78afa995795e4d85b5d9ceeca43f5fef|offer received|0.0                  |{NULL, 9b98b8c7a33c4b65b9aebfe6a799e6d9, NULL, NULL}|
|a03223e636434f42ac4c3df47e8bac43|offer received|0.0                  |{NULL, 0b1e1539f2cc45b7b9fa7c272da2e1d7, NULL, NULL}|
|e2127556f4f64592b11af22de27a7932|offer received|0.0                  |{NULL, 2906b810c7d4411798c6938adc9daaa5, NULL, NULL}|
|8ec6ce2a7e7949b1bf142def7d0e0586|offer received|0.0                  |{NULL, fafdcd668e3743c1bb461111dcafc2a4, NULL, NULL}|
|68617ca6246f4fbc85e91a2a49552598|offer received|0.0                  |{NULL, 4d5c57ea9a6940dd891ad53e9dbe8da0, NULL, NULL}|


In [29]:
transactions.groupBy('event').count().orderBy('count',ascending=False).show(10,False)

+---------------+------+
|event          |count |
+---------------+------+
|transaction    |138953|
|offer received |76277 |
|offer viewed   |57725 |
|offer completed|33579 |
+---------------+------+



In [31]:
transactions.orderBy('account_id',ascending=False).show(20,False)

+--------------------------------+---------------+---------------------+----------------------------------------------------+
|account_id                      |event          |time_since_test_start|value                                               |
+--------------------------------+---------------+---------------------+----------------------------------------------------+
|ffff82501cea40309d5fdd7edcca4a07|offer received |0.0                  |{NULL, fafdcd668e3743c1bb461111dcafc2a4, NULL, NULL}|
|ffff82501cea40309d5fdd7edcca4a07|offer viewed   |0.25                 |{NULL, fafdcd668e3743c1bb461111dcafc2a4, NULL, NULL}|
|ffff82501cea40309d5fdd7edcca4a07|transaction    |5.0                  |{9.12, NULL, NULL, NULL}                            |
|ffff82501cea40309d5fdd7edcca4a07|transaction    |2.5                  |{16.06, NULL, NULL, NULL}                           |
|ffff82501cea40309d5fdd7edcca4a07|offer completed|2.5                  |{NULL, NULL, fafdcd668e3743c1bb461111dcafc2a4,

In [28]:
transactions.filter('value.amount is not null').show(10,False)

+--------------------------------+-----------+---------------------+-------------------------+
|account_id                      |event      |time_since_test_start|value                    |
+--------------------------------+-----------+---------------------+-------------------------+
|02c083884c7d45b39cc68e1314fec56c|transaction|0.0                  |{0.83, NULL, NULL, NULL} |
|9fa9ae8f57894cc9a3b8a9bbe0fc1b2f|transaction|0.0                  |{34.56, NULL, NULL, NULL}|
|54890f68699049c2a04d415abc25e717|transaction|0.0                  |{13.23, NULL, NULL, NULL}|
|b2f1cd155b864803ad8334cdf13c4bd2|transaction|0.0                  |{19.51, NULL, NULL, NULL}|
|fe97aa22dd3e48c8b143116a8403dd52|transaction|0.0                  |{18.97, NULL, NULL, NULL}|
|629fc02d56414d91bca360decdfa9288|transaction|0.0                  |{33.9, NULL, NULL, NULL} |
|bbeb54e861614fc7b22a8844f72dca6c|transaction|0.0                  |{0.22, NULL, NULL, NULL} |
|a97e6f33219c432db82acfa0d19c602d|transaction|0.0 

In [26]:
offers.show(10,False)




+----------------------------+--------------+--------+--------------------------------+---------+-------------+
|channels                    |discount_value|duration|id                              |min_value|offer_type   |
+----------------------------+--------------+--------+--------------------------------+---------+-------------+
|[email, mobile, social]     |10            |7.0     |ae264e3637204a6fb9bb56bc8210ddfd|10       |bogo         |
|[web, email, mobile, social]|10            |5.0     |4d5c57ea9a6940dd891ad53e9dbe8da0|10       |bogo         |
|[web, email, mobile]        |0             |4.0     |3f207df678b143eea3cee63160fa8bed|0        |informational|
|[web, email, mobile]        |5             |7.0     |9b98b8c7a33c4b65b9aebfe6a799e6d9|5        |bogo         |
|[web, email]                |5             |10.0    |0b1e1539f2cc45b7b9fa7c272da2e1d7|20       |discount     |
|[web, email, mobile, social]|3             |7.0     |2298d6c36e964ae4a3e7e9706d1fb8c2|7        |discoun

In [3]:
profile = ProfileReport(offers, title="Offers Report")

In [5]:
profile.to_html()


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]


ValueError: setting an array element with a sequence

In [27]:
offers.show(20,False)

+----------------------------+--------------+--------+--------------------------------+---------+-------------+
|channels                    |discount_value|duration|id                              |min_value|offer_type   |
+----------------------------+--------------+--------+--------------------------------+---------+-------------+
|[email, mobile, social]     |10            |7.0     |ae264e3637204a6fb9bb56bc8210ddfd|10       |bogo         |
|[web, email, mobile, social]|10            |5.0     |4d5c57ea9a6940dd891ad53e9dbe8da0|10       |bogo         |
|[web, email, mobile]        |0             |4.0     |3f207df678b143eea3cee63160fa8bed|0        |informational|
|[web, email, mobile]        |5             |7.0     |9b98b8c7a33c4b65b9aebfe6a799e6d9|5        |bogo         |
|[web, email]                |5             |10.0    |0b1e1539f2cc45b7b9fa7c272da2e1d7|20       |discount     |
|[web, email, mobile, social]|3             |7.0     |2298d6c36e964ae4a3e7e9706d1fb8c2|7        |discoun

In [32]:
profiles.show(10,False)

+---+-----------------+------+--------------------------------+-------------+
|age|credit_card_limit|gender|id                              |registered_on|
+---+-----------------+------+--------------------------------+-------------+
|118|NULL             |NULL  |68be06ca386d4c31939f3a4f0e3dd783|20170212     |
|55 |112000.0         |F     |0610b486422d4921ae7d2bf64640c50b|20170715     |
|118|NULL             |NULL  |38fe809add3b4fcf9315a9694bb96ff5|20180712     |
|75 |100000.0         |F     |78afa995795e4d85b5d9ceeca43f5fef|20170509     |
|118|NULL             |NULL  |a03223e636434f42ac4c3df47e8bac43|20170804     |
|68 |70000.0          |M     |e2127556f4f64592b11af22de27a7932|20180426     |
|118|NULL             |NULL  |8ec6ce2a7e7949b1bf142def7d0e0586|20170925     |
|118|NULL             |NULL  |68617ca6246f4fbc85e91a2a49552598|20171002     |
|65 |53000.0          |M     |389bc3fa690240e798340f5a15918d5c|20180209     |
|118|NULL             |NULL  |8974fc5686fe429db53ddde067b88302|2

In [33]:
profilespd = profiles.toPandas()